4. When done, write a script to count the fraction of ham and spam predictions.  Compare to the fraction in the Minio logs – do they agree?

In [21]:
import datetime
import json
import os
import boto3
from dotenv import load_dotenv
from datetime import datetime
from flask import Flask, request, jsonify
from botocore.errorfactory import ClientError
import structlog  # for event logging
import pandas as pd
import numpy as np
import os
import json
import glob
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import scipy
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.cluster import DBSCAN
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import joblib

In [22]:

# load the environment files
load_dotenv()
s3_resource = boto3.resource('s3',
                             endpoint_url=os.getenv('ENDPOINT_URL'),
                             aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
                             aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
                             aws_session_token=None,
                             config=boto3.session.Config(signature_version='s3v4'),
                             verify=False
                             )
s3_client = boto3.client('s3',
                             endpoint_url=os.getenv('ENDPOINT_URL'),
                             aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
                             aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
                             aws_session_token=None,
                             config=boto3.session.Config(signature_version='s3v4'),
                             verify=False
                             )

In [23]:
def read_from_s3_bucket(file_name: str, bucket_name="prediction-log-files"):
    # read all the files in the bucket
    bucket = s3_resource.Bucket(bucket_name)
    found = False
    data = ""
    for obj in bucket.objects.all():
        key = obj.key
        if file_name in str(key):
            body = obj.get()['Body'].read()
            
            if isinstance(body, bytes):
                body = body.decode()
            elif not isinstance(body, str):
                body = str(body)
                
            print("%s : length = %s" % (key, len(body)))
                
            data += body
            # data = body
            # data.append(body)
            found = True
    if found:
        return data
    else:
        print("ERROR KEY NOT FOUND")
        return -1

In [24]:
DEBUG = True
# read in all the data from S3 as one big string
data = read_from_s3_bucket(file_name="log_file")

log_file_20230203-155922.json : length = 16648
log_file_20230203-160023.json : length = 55461
log_file_20230203-160124.json : length = 55520
log_file_20230203-160224.json : length = 55880
log_file_20230203-160325.json : length = 55643
log_file_20230203-160425.json : length = 55637
log_file_20230203-160526.json : length = 55712
log_file_20230203-160626.json : length = 55063
log_file_20230203-160727.json : length = 54372
log_file_20230203-160827.json : length = 54524
log_file_20230203-160928.json : length = 54008
log_file_20230203-161028.json : length = 54875
log_file_20230203-161129.json : length = 54534
log_file_20230203-161229.json : length = 51869
log_file_20230203-161330.json : length = 46333
log_file_20230203-161430.json : length = 53554
log_file_20230203-161531.json : length = 54479
log_file_20230203-161631.json : length = 48506
log_file_20230203-161732.json : length = 44613
log_file_20230203-161833.json : length = 53580
log_file_20230203-161933.json : length = 54117
log_file_2023

In [25]:
# Split the string of data into individual lines. Only works because backslash is escaped in string.
data = data.split('\n')
data = data[0:-1]
if DEBUG: print(len(data))
if DEBUG: print(data[0])

# convert every entry in the list to a JSON object. 
records = []
for i in range(len(data)):
    record = json.loads(data[i])
    records.append(record)

if DEBUG: print("Type: %s of type: %s, Len %d" % (type(records), type(records[0]), len(records)))
if DEBUG: print(json.dumps(records[0]))

10000
{"predicted_class": "spam", "event": "classify_email:predicted_class", "timestamp": "2023-02-03T21:59:04.887237Z"}
Type: <class 'list'> of type: <class 'dict'>, Len 10000
{"predicted_class": "spam", "event": "classify_email:predicted_class", "timestamp": "2023-02-03T21:59:04.887237Z"}


In [26]:
# load the data into a dataframe and set proper types
df = pd.DataFrame(records)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['predicted_class'] = df['predicted_class'].astype('category')
df['event'] = df['event'].astype('category')
df = df.sort_values(by='timestamp')

if DEBUG: print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   predicted_class  10000 non-null  category           
 1   event            10000 non-null  category           
 2   timestamp        10000 non-null  datetime64[ns, UTC]
dtypes: category(2), datetime64[ns, UTC](1)
memory usage: 176.0 KB
None


In [27]:
ham = df[df['predicted_class']=='ham']
spam = df[df['predicted_class']=='spam']
n_ham = len(ham)
n_spam = len(spam)
print("n_ham: %d\tn_spam: %d" % (n_ham, n_spam))

n_ham: 2786	n_spam: 7214


In [28]:
pct_ham = n_ham / n_spam * 100
print("The logs predict %.2f percent ham" % pct_ham)

The logs predict 38.62 percent ham
